In [2]:
#Importing libraries
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D
from tensorflow.keras import Model
import PIL
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator #allows us to augment and process data fed into a CNN
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os,cv2,random
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator #A tool that allows us to augment and process data to be fed into a CNN
from sklearn.model_selection import KFold
import numpy as np

In [9]:
path_inception = "D:/amitavsir/bcd/data/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
local_weights_file = path_inception
pre_trained_model = VGG16(input_shape = (121,121,3),
              include_top = False,
              weights = None)
pre_trained_model.load_weights(local_weights_file)

In [10]:
#Locking the weights and parameters of pretrained model
for layer in pre_trained_model.layers:
    layer.trainable = False

#Getting desired layer output
last_layer = pre_trained_model.get_layer('block5_conv2')
last = last_layer.output
print('Last layer output shape : ', last.shape)

Last layer output shape :  (?, 7, 7, 512)


In [11]:
from tensorflow.keras.losses import categorical_crossentropy
# Model configuration
batch_size = 50
img_size, img_num_channels = 121, 3
loss_function = categorical_crossentropy
no_classes = 4
no_epochs = 20
optimizer = Adam()
verbosity = 1

In [12]:
data_path='D:/amitavsir/bcd/datacompressed/train'
list_folder=os.listdir(path = data_path)
data=[]
im_size=121    
for i in list_folder:
    new_path=os.path.join(data_path,i) 
    pic_list=os.listdir(new_path)                                               
    for img in pic_list:
        pic=os.path.join(new_path,img)   
        arr=cv2.imread(pic)    
        data.append([arr,list_folder.index(i)])    
        
random.shuffle(data)  
x_train,y_train=[],[]
for i,j in data:
    x_train.append(i)
    y_train.append(j)
x_train=np.array(x_train).reshape(-1,im_size,im_size,img_num_channels)
y_train=np.array(y_train).reshape(-1,1)

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train)
X_train = x_train/255
y_train = y_train.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X_train,y_train, test_size=0.25)

In [13]:
# Parse numbers as floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [14]:
# Define per-fold score containers <-- these are new
acc_per_fold = [] #stores accuracy
loss_per_fold = [] #stores losses

In [15]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

In [16]:
#class for training set
train_data = ImageDataGenerator(rescale = 1./255, #pixel values rescaled so that it stays between 0 and 1.
                                shear_range = 0.2, #this and the next 4 steps are used to augment our data by shearing it, flipping and zooming it to produce more examples per image.
                                zoom_range = 0.2,  #Data augmentation allows us to prevent overfitting of data into the training set.
                                horizontal_flip = True,
                                vertical_flip = True,
                                rotation_range = 40,
                                width_shift_range = 0.2,
                                height_shift_range = 0.2,
                                fill_mode = 'nearest'
                                )
test_data = ImageDataGenerator(rescale = 1/.255)

train_set = train_data.flow_from_directory('D:/amitavsir/bcd/datacompressed/train', #Image path
                                           target_size = (im_size,im_size), #The shape which we want to input our images in our model
                                           batch_size = 20,         #The batch size in mini batch gradient descent
                                           class_mode = 'categorical')   


test_set = test_data.flow_from_directory('D:/amitavsir/bcd/datacompressed/validation',
                                         target_size = (im_size,im_size),
                                         batch_size = 50,
                                         class_mode = 'categorical')

Found 962 images belonging to 4 classes.
Found 500 images belonging to 4 classes.


In [24]:
num_folds=10

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1

In [25]:
for train, test in kfold.split(inputs, targets):
    #Building own model on top of trained network
    x = Conv2D(500, (1,1), activation = 'relu')(last)
    x = Flatten()(x)
    x = Dense(1028, activation = 'relu')(x)
    x = Dropout(rate = 0.2)(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dropout(rate = 0.25)(x)
    x = Dense(4, activation = 'softmax')(x)

    #Compiling model
    model = Model(inputs = pre_trained_model.input, outputs = x, name = 'Predict')
    opt1 = Adam(learning_rate = 0.001)
    opt2 = RMSprop(learning_rate = 0.001)
    model.compile(optimizer = opt1 , loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,
              #validation_data = test_set,
              #validation_steps = 5          
              )
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/20
865/865 [==============================] - 74s 85ms/sample - loss: 4.6534 - acc: 0.4231
Epoch 2/20
865/865 [==============================] - 76s 87ms/sample - loss: 0.8507 - acc: 0.6520
Epoch 3/20
865/865 [==============================] - 79s 91ms/sample - loss: 0.6051 - acc: 0.7584
Epoch 4/20
865/865 [==============================] - 77s 89ms/sample - loss: 0.4473 - acc: 0.8347
Epoch 5/20
865/865 [==============================] - 99s 114ms/sample - loss: 0.3476 - acc: 0.8763
Epoch 6/20
865/865 [==============================] - 113s 130ms/sample - loss: 0.2814 - acc: 0.8867
Epoch 7/20
865/865 [==============================] - 114s 132ms/sample - loss: 0.2082 - acc: 0.9306
Epoch 8/20
865/865 [==============================] - 110s 127ms/sample - loss: 0.1638 - acc: 0.9422
Epoch 9/20
865/865 [==============================] - 119s 138ms/sample - loss: 0.1426 - acc: 0.9503
Epo

In [26]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 21.081590298524837 - Accuracy: 38.14432919025421%
------------------------------------------------------------------------
> Fold 2 - Loss: 21.368139070333893 - Accuracy: 23.711340129375458%
------------------------------------------------------------------------
> Fold 3 - Loss: 21.17801030476888 - Accuracy: 25.0%
------------------------------------------------------------------------
> Fold 4 - Loss: 21.301259358723957 - Accuracy: 28.125%
------------------------------------------------------------------------
> Fold 5 - Loss: 21.314400990804035 - Accuracy: 15.625%
------------------------------------------------------------------------
> Fold 6 - Loss: 21.138458887736004 - Accuracy: 25.0%
------------------------------------------------------------------------
> Fold 7 - Loss: 21.196800867716473 - Accuracy: